### `None` in sys.modules

In [1]:
import sys
sys.modules['module'] = None

import module

ModuleNotFoundError: import of module halted; None in sys.modules

### `sys.meta_path` exhausted

In [2]:
del sys.modules['module']
del sys.meta_path[:]

import module

ModuleNotFoundError: No module named 'module'

### Spec's loader is `None`

In [7]:
from importlib.abc import MetaPathFinder
from importlib.machinery import ModuleSpec

class NoLoaderFinder(MetaPathFinder):
    def find_spec(self, fullname, path, target=None):
        return ModuleSpec('module', None)

finder = NoLoaderFinder()
sys.meta_path = [finder]

import module

ImportError: missing loader

### Regular loader

In [17]:
from types import ModuleType
from importlib.abc import Loader

class Module(ModuleType):
    def __init__(self, name):
        self.x = 1
        self.name = name

class ExampleLoader(Loader):
    def create_module(self, spec):
        return Module(spec.name)

    def exec_module(self, module):
        module.y = 2

class ExampleFinder(MetaPathFinder):
    def find_spec(self, fullname, path, target=None):
        return ModuleSpec('module', ExampleLoader())

sys.meta_path = [ExampleFinder()]

import module
print('module', module)
print('module.name', module.name)
print('module.x', module.x)
print('module.y', module.y)

del sys.modules['module']

print('-' * 10)
module2 = __import__('module')

print('module2 is module?', module2 is module)
print('module2', module2)
print('module2.name', module2.name)
print('module2.x', module2.x)
print('module2.y', module2.y)



module <module 'module' (<__main__.ExampleLoader object at 0x7f782ed6e3d0>)>
module.name module
module.x 1
module.y 2
----------
module2 is module? False
module2 <module 'module' (<__main__.ExampleLoader object at 0x7f782deab5d0>)>
module2.name module
module2.x 1
module2.y 2
